In [ ]:
library(arrow)
library(aws.s3)
library(haven) # read_sav
library(labelled)

# Configuração do S3
Sys.setenv(
    "AWS_ACCESS_KEY_ID" = "",
    "AWS_SECRET_ACCESS_KEY" = "",
    "AWS_S3_ENDPOINT"= "s3-h.ipe.df.gov.br"
)

OBJECT_OLD_KEY = 'DIEPS/bases/ped/NovaPEDDF2016.sav'
OBJECT_NEW_KEY = 'DIEPS/bases/ped-parquet/NovaPEDDF2016.parquet'
BUCKET_NAME = 'raw'
OUTPUT_NAME = '/tmp/output.parquet'

In [33]:
# Verifica conexão com o MinIO
bucketlist(region = '', use_https = TRUE)

,Bucket,CreationDate
,<chr>,<chr>
1,archive,2025-04-04T13:55:16.749Z
2,intermediate,2024-10-21T14:53:31.843Z
3,mart,2024-10-17T12:22:09.832Z
4,raw,2024-10-21T14:53:16.576Z
5,teste-admin,2024-12-05T17:42:15.673Z


In [34]:
# Busca o objeto
tempfile = tempfile(fileext = '.sav')
object_path = paste0('s3://', BUCKET_NAME, '/', OBJECT_OLD_KEY)
filename = save_object(object = object_path, file = tempfile, region = '', use_https = TRUE)
filename

[1] "/tmp/RtmpSLuxXB/file5a4e26f58589c.sav"

In [35]:
data <- read_sav(filename)
head(data)

DOMIC,FAMILIA,PESSOA,GRUPO,CONGLOM,AAMM,FATOR,C000,C030,C040,⋯,SIT,POS,CNAE,SETOR_CNAE,INST,INFLATORDF,SM,TOTFAM,TOTMORA,TAMFAM
<dbl>,<dbl>,<dbl>,<dbl+lbl>,<dbl>,<dbl>,<dbl>,<dbl+lbl>,<dbl+lbl>,<dbl+lbl>,⋯,<dbl+lbl>,<dbl+lbl>,<dbl+lbl>,<dbl+lbl>,<dbl+lbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
240020,1,1,1,2.8e+10,201607,964.6938,1,1,1,⋯,4,7,5200,5000,9,275.12,880,1,3,3
240020,1,2,1,2.8e+10,201607,964.6938,1,2,1,⋯,4,3,5400,5000,9,275.12,880,1,3,3
240020,1,3,1,2.8e+10,201607,964.6938,1,1,1,⋯,4,12,5200,5000,9,275.12,880,1,3,3
240070,1,1,1,2.8e+10,201607,964.6938,1,2,1,⋯,4,2,5200,5000,10,275.12,880,1,2,2
240070,1,2,1,2.8e+10,201607,1138.9987,1,1,1,⋯,10,0,10000,10000,4,275.12,880,1,2,2
240090,1,1,1,2.8e+10,201607,947.3296,2,1,1,⋯,6,0,10000,10000,9,275.12,880,1,3,3


# Substituição de labels

As labels geradas pelo SPSS não tem um formato legal de serialização, então antes de armazenar os dados em parquet, seria interesante adicionar os dados em um formato de melhor serialização, como um dicionário.

In [42]:
# Converte os campos categóricos em factor
data = as_factor(data, only_labelled = TRUE)
head(data)

DOMIC,FAMILIA,PESSOA,GRUPO,CONGLOM,AAMM,FATOR,C000,C030,C040,⋯,SIT,POS,CNAE,SETOR_CNAE,INST,INFLATORDF,SM,TOTFAM,TOTMORA,TAMFAM
<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<fct>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
240020,1,1,Alta Renda,2.8e+10,201607,964.6938,Apenas um morador,Masculino,Branca,⋯,Ocupado,Empregador,"Informação e Comunicação; Ativid. Financeiras, Seguros e Relacionados, Ativid.Profissionais, Científicas e Técnica",Serviços,Ensino Superior Completo,275.12,880,1,3,3
240020,1,2,Alta Renda,2.8e+10,201607,964.6938,Apenas um morador,Feminino,Branca,⋯,Ocupado,Ass Pub,"Administração Pública, Defesa e Seguridade Social, Educação, Saúde Humana e Serviços Sociais",Serviços,Ensino Superior Completo,275.12,880,1,3,3
240020,1,3,Alta Renda,2.8e+10,201607,964.6938,Apenas um morador,Masculino,Branca,⋯,Ocupado,Outros,"Informação e Comunicação; Ativid. Financeiras, Seguros e Relacionados, Ativid.Profissionais, Científicas e Técnica",Serviços,Ensino Superior Completo,275.12,880,1,3,3
240070,1,1,Alta Renda,2.8e+10,201607,964.6938,Apenas um morador,Feminino,Branca,⋯,Ocupado,Ass Priv S Cart,"Informação e Comunicação; Ativid. Financeiras, Seguros e Relacionados, Ativid.Profissionais, Científicas e Técnica",Serviços,Pós graduação,275.12,880,1,2,2
240070,1,2,Alta Renda,2.8e+10,201607,1138.9987,Apenas um morador,Masculino,Branca,⋯,Não se aplica (menor de 14 anos),Não ocupado,Não ocupado,Não ocupado,Ensino Fundamental Incompleto,275.12,880,1,2,2
240090,1,1,Alta Renda,2.8e+10,201607,947.3296,Mais de um morador,Masculino,Branca,⋯,Inativo Puro,Não ocupado,Não ocupado,Não ocupado,Ensino Superior Completo,275.12,880,1,3,3


In [43]:
# Converte o dado para uma tabela do arrow
df2 <- arrow_table(data) 
df2

Table
29773 rows x 152 columns
$DOMIC <double>
$FAMILIA <double>
$PESSOA <double>
$GRUPO <dictionary<values=string, indices=int8>>
$CONGLOM <double>
$AAMM <double>
$FATOR <double>
$C000 <dictionary<values=string, indices=int8>>
$C030 <dictionary<values=string, indices=int8>>
$C040 <dictionary<values=string, indices=int8>>
$C050 <dictionary<values=string, indices=int8>>
$C061 <dictionary<values=string, indices=int8>>
$C062 <dictionary<values=string, indices=int8>>
$C070 <dictionary<values=string, indices=int8>>
$D010 <dictionary<values=string, indices=int8>>
$D0211 <dictionary<values=string, indices=int8>>
$D0212 <dictionary<values=string, indices=int16>>
$D0221 <dictionary<values=string, indices=int8>>
$D0222 <dictionary<values=string, indices=int8>>
$D0231 <dictionary<values=string, indices=int8>>
...
132 more columns
Use `schema()` to see entire schema

See $metadata for additional Schema metadata

# Conversão para parquet

Converte o dataframe para parquet

In [40]:
write_parquet(
    x = df2,
    sink = OUTPUT_NAME,
    compression = 'snappy',
    use_dictionary = TRUE
)

# Carregamento

Salva o arquivo de volta no bucket `raw`.

In [41]:
put_object(
    bucket = BUCKET_NAME,
    object = OBJECT_NEW_KEY,
    file = OUTPUT_NAME,
    region = '',
    use_https = TRUE
)

[1] TRUE